In [1]:
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from cdp_backend.pipeline import ingestion_models
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd

In [2]:
# Get basic council memeber information
city_council = pd.DataFrame()
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
council_url = 'https://www.minneapolismn.gov/government/city-council/'
driver.get(council_url)
cards = driver.find_elements(By.CSS_SELECTOR, '#image-text-link-50242 a.card-link')
len(cards)
for card in cards:
    driver.get(card.get_attribute('href'))
    position, name = driver.find_element(By.CSS_SELECTOR, '.masthead--title').text.split(' - ')
    role = driver.find_element(By.CSS_SELECTOR, 'a.contact span.title').text
    city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
    driver.back()
driver.quit()

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\3421782549.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  city_council = city_council.append({'name': name, 'position': position, 'role': role}, ignore_index=True)
C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\3421782549.py:12: FutureWarning: The frame.append method is deprecated and will be removed from

In [3]:
city_council

,name,position,role
0,Elliott Payne,Ward 1,Council Member
1,Robin Wonsley,Ward 2,Council Member
2,Michael Rainville,Ward 3,Council Member
3,LaTrisha Vetaw,Ward 4,Council Member
4,Jeremiah Ellison,Ward 5,Council Member
5,Jamal Osman,Ward 6,Council Member
6,Lisa Goodman,Ward 7,Council Member
7,Andrea Jenkins,Ward 8,Council President
8,Jason Chavez,Ward 9,Council Member
9,Aisha Chughtai,Ward 10,Council Member


In [129]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)
dt.isoformat()
dt.year, dt.month, dt.day
# video_link = driver.find_element(by='id', value='VideoIcon_0_0')
# video_link.click()
# video = WebDriverWait(driver, 4).until(
#     ec.presence_of_element_located((By.CSS_SELECTOR, '.close span'))
# )

# link = WebDriverWait(driver, 2).until(
#     ec.presence_of_element_located((By.CSS_SELECTOR, 'iframe'))
# )
# sessions.append(
#     ingestion_models.Session(
#         # Get Video Link:
#         video_uri=link.get_attribute('src'),
#         session_index=0,
#         session_datetime=dt
#     )
# )

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_20992\200673377.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


ValueError: not enough values to unpack (expected 5, got 1)

In [5]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)

sessions = []

# Get Datetime:
dt = driver.find_element(by='css selector', value='.agendaHeader .col-md-12:nth-child(3)').text \
    .replace(',', '').replace('- ', '').split(' ')
print(dt)
month, day, year, time, ampm = dt
hour, minute = time.split(':')
hour, minute = int(hour), int(minute)
hour, minute
if ampm == 'pm':
    hour += 12
dt = datetime(int(year), datetime.strptime(month, "%B").month, int(day), hour, minute)
dt.isoformat()
dt.year, dt.month, dt.day

# Get voters
member_roles = {}
for member in city_council['name']:
    member_roles[member] = city_council[city_council['name'] == member]['role']
members = driver.find_element(By.XPATH, '//*[@id="markedAgendaSection"]/div[1]/div/div/div[2]')
members = members.text.replace('Members Present: Council Members ', '').split(', ')
members[-1] = members[-1].replace('and ', '')
for member in members:
    member = member.split(' ')
    if len(member) > 2:
        member_roles[member[0]+' '+member[1]] = 'Council ' + member[2].replace('(', '').replace(')', '')
driver.quit()
member_roles

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\1607318454.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


['June', '30', '2022', '9:30', 'am']


{'Elliott Payne': 0    Council Member
 Name: role, dtype: object,
 'Robin Wonsley': 1    Council Member
 Name: role, dtype: object,
 'Michael Rainville': 2    Council Member
 Name: role, dtype: object,
 'LaTrisha Vetaw': 3    Council Member
 Name: role, dtype: object,
 'Jeremiah Ellison': 4    Council Member
 Name: role, dtype: object,
 'Jamal Osman': 5    Council Member
 Name: role, dtype: object,
 'Lisa Goodman': 6    Council Member
 Name: role, dtype: object,
 'Andrea Jenkins': 'Council President',
 'Jason Chavez': 8    Council Member
 Name: role, dtype: object,
 'Aisha Chughtai': 9    Council Member
 Name: role, dtype: object,
 'Emily Koski': 10    Council Member
 Name: role, dtype: object,
 'Andrew Johnson': 11    Council Member
 Name: role, dtype: object,
 'Linea Palmisano': 'Council Vice-President'}

In [126]:
driver.quit()

In [118]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)
li = driver.find_elements(by='css selector', value='li.customLineHeight')
for list in li[:9]:
    try:
        if list.find_element(By.CSS_SELECTOR, 'a').get_attribute('class') == 'hrefnowrap':
            link = list.find_element(By.CSS_SELECTOR, 'a.hrefnowrap')
            print(link.text)
    except (selenium.common.exceptions.NoSuchElementException):
        continue
driver.quit()

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\104466363.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


This link open a new window
(2022-00585)
This link open a new window
(2022-00559)


In [127]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = "https://lims.minneapolismn.gov/MarkedAgenda/3323"
driver.get(url)


# Inspecting each minute_item:
li = driver.find_elements(by='css selector', value='li.customLineHeight')
event_minutes_items = []
month_convert = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}
for list in li:
    try:
        if list.find_element(By.CSS_SELECTOR, 'a').get_attribute('class') == 'hrefnowrap':
            link = list.find_element(By.CSS_SELECTOR, 'a.hrefnowrap')
            link.click()
            before = driver.window_handles[0]
            after = driver.window_handles[1]
            driver.switch_to.window(after)
            element = WebDriverWait(driver, 2).until(
                ec.visibility_of_element_located((By.CSS_SELECTOR, 'body'))
            )
            print('We are currently in the website: ' + driver.current_url)
            togglets = element.find_elements(By.CSS_SELECTOR, '.togglet')
            
            
            
            '''
            If toggles are closed go into this for loop!
            '''
            if len(togglets) > 1:
                for i, togglet in enumerate(togglets):
                    togglets_new = WebDriverWait(driver, 5).until(
                        ec.visibility_of_all_elements_located((By.CSS_SELECTOR, '.togglet'))
                    )
                    togglet = togglets_new[i]
                    
                    togglec = togglet.find_element(By.XPATH, "./following-sibling::div")
                    print('The togglet context: ' + togglet.text)
                    m_decision = togglet.find_element(By.CSS_SELECTOR, 'a span.detail_Status')
                    print('decision: ' + m_decision.text)
                    if m_decision != 'Hearing Scheduled':
                        togglet.click()
                    with open(f'toggle-{i}.txt', 'w') as open_f:
                        open_f.write(togglec.text)
                    print(f"wrote text content of togglec: {i}")
                    minutes_item = togglec.find_element(By.CSS_SELECTOR, 'a:nth-child(1)')
                    print('name: ' + minutes_item.text)
                    m_title = togglet.find_element(By.CSS_SELECTOR, '.legisDetails-tog').text
                    print('title: ' + m_title)

                    
                    h5s = togglec.find_elements(By.CSS_SELECTOR, 'h5')
                    m_type = ''
                    for h5 in h5s:
                        if ('Effective Date:' in h5.text):
                            m_type = h5.text.split(' ')[0]
                    print('matter type: ' + m_type)
                    table = togglec.find_element(By.CSS_SELECTOR, '.col_full.nobottommargin table')
                    m_votes = []
                    rows = togglec.find_elements(By.CSS_SELECTOR, 'tbody tr')
                    rows = rows[:-1]
                    for j, row in enumerate(rows):
                        vote_date = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text #.split(' ')
                        # print('The date is: ' + vote_date)
                        vote_date = vote_date.replace(vote_date[:3], month_convert[vote_date[:3]])
                        vote_date = datetime.strptime(vote_date, "%B %d, %Y")
                        print('date parsed!')
                        if (vote_date.year, vote_date.month, vote_date.day) == (dt.year, dt.month, dt.day):
                            print('We got the matched vote date: ' + str(vote_date))
                            vote_link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a')
                            if vote_link.text == 'View Voting':
                                vote_link.click()
                                print('clicking into the voting tab')
                                voting_details = WebDriverWait(driver, 5).until(
                                    ec.visibility_of_element_located((By.XPATH, f'//*[@id="model_{i}_{j}"]'))
                                )

                                print('start finding voters!')
                                
                                voters = voting_details.find_elements(By.CSS_SELECTOR, '.vote-row')
                                for voter in voters:
                                    voter_name = voter.find_element(By.CSS_SELECTOR, '.vote-cell:nth-child(1)').text
                                    print('voter name: ' + voter_name)
                                    voter_decision = voter.find_element(By.CSS_SELECTOR, '.vote-cell:nth-child(2)').text
                                    voter_is_active = voter_name in city_council.name.tolist()
                                    voter_position = city_council[city_council['name'] == voter_name]['position']
                                    voter_role = member_roles[voter_name]
                                    m_votes.append(
                                        ingestion_models.Vote(
                                            person=ingestion_models.Person(
                                                voter_name,
                                                is_active = voter_is_active,
                                                seat=ingestion_models.Seat(
                                                    voter_position,
                                                    roles = ingestion_models.Role(voter_role)
                                                )
                                            ),
                                            decision = voter_decision
                                        )
                                    )
                                
                                print('adding into event_minutes_items!')
                                event_minutes_items.append(
                                    ingestion_models.EventMinutesItem(
                                        minutes_item=ingestion_models.MinutesItem(minutes_item.text),
                                        matter = ingestion_models.Matter(
                                            name = minutes_item.text,
                                            title = m_title,
                                            matter_type=m_type
                                        ),
                                        decision=m_decision,
                                        votes = m_votes
                                    )
                                )
                                
                                close_button = voting_details.find_element(By.XPATH, f'//*[@id="model_{str(i)}_{j}"]/div/div/div/div[1]/button')
                                # print('The button shows like: ' + close_button.text)
                                close_button.click()
                                print('Go out!')
                                break
                        
                    print("-" * 80)
            # If toggles are opened, then do the following:
            else:      
                togglet = element.find_element(By.CSS_SELECTOR, '.togglet')
                togglec = togglet.find_element(By.XPATH, "./following-sibling::div")
                print('The togglet context: ' + togglet.text)
                m_decision = togglet.find_element(By.CSS_SELECTOR, 'a span.detail_Status')
                print('decision: ' + m_decision.text)
                minutes_item = togglec.find_element(By.CSS_SELECTOR, 'a:nth-child(1)')
                print('name: ' + minutes_item.text)
                m_title = togglet.find_element(By.CSS_SELECTOR, '.legisDetails-tog').text
                print('title: ' + m_title)

                
                h5s = togglec.find_elements(By.CSS_SELECTOR, 'h5')
                m_type = ''
                for h5 in h5s:
                    if ('Effective Date:' in h5.text):
                        m_type = h5.text.split(' ')[0]
                print('matter type: ' + m_type)
                table = togglec.find_element(By.CSS_SELECTOR, '.col_full.nobottommargin table')
                m_votes = []
                rows = togglec.find_elements(By.CSS_SELECTOR, 'tbody tr')
                rows = rows[:-1]
                for j, row in enumerate(rows):
                    vote_date = row.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text #.split(' ')
                    # print('The date is: ' + vote_date)
                    vote_date = vote_date.replace(vote_date[:3], month_convert[vote_date[:3]])
                    vote_date = datetime.strptime(vote_date, "%B %d, %Y")
                    print('date parsed!')
                    if (vote_date.year, vote_date.month, vote_date.day) == (dt.year, dt.month, dt.day):
                        print('We got the matched vote date: ' + str(vote_date))
                        vote_link = row.find_element(By.CSS_SELECTOR, 'td:nth-child(2) a')
                        if vote_link.text == 'View Voting':
                            vote_link.click()
                            print('clicking into the voting tab')
                            voting_details = WebDriverWait(driver, 5).until(
                                ec.visibility_of_element_located((By.XPATH, f'//*[@id="model_0_{j}"]'))
                            )

                            print('start finding voters!')
                                
                            voters = voting_details.find_elements(By.CSS_SELECTOR, '.vote-row')
                            for voter in voters:
                                voter_name = voter.find_element(By.CSS_SELECTOR, '.vote-cell:nth-child(1)').text
                                # print('voter name: ' + voter_name)
                                voter_decision = voter.find_element(By.CSS_SELECTOR, '.vote-cell:nth-child(2)').text
                                voter_is_active = voter_name in city_council.name.tolist()
                                voter_position = city_council[city_council['name'] == voter_name]['position']
                                voter_role = member_roles[voter_name]
                                m_votes.append(
                                    ingestion_models.Vote(
                                        person=ingestion_models.Person(
                                            voter_name,
                                            is_active = voter_is_active,
                                            seat=ingestion_models.Seat(
                                                voter_position,
                                                roles = ingestion_models.Role(voter_role)
                                            )
                                        ),
                                        decision = voter_decision
                                    )
                                )
                            
                            print('adding into event_minutes_items!')
                            event_minutes_items.append(
                                ingestion_models.EventMinutesItem(
                                    minutes_item=ingestion_models.MinutesItem(minutes_item.text),
                                    matter = ingestion_models.Matter(
                                        name = minutes_item.text,
                                        title = m_title,
                                        matter_type=m_type
                                    ),
                                    decision=m_decision,
                                    votes = m_votes
                                )
                            )
                            
                            close_button = voting_details.find_element(By.XPATH, f'//*[@id="model_0_{j}"]/div/div/div/div[1]/button')
                            # print('The button shows like: ' + close_button.text)
                            close_button.click()
                            print('Go out!')
                            break
                print("-" * 80)
            
                       
            # raise(ArithmeticError)       
                    
            driver.close()
            print('Closing the current tab')
            driver.switch_to.window(before)
            print('Switching to the other tab')
            
        else:
            try:
                model = ingestion_models.EventMinutesItem(
                    minutes_item = ingestion_models.MinutesItem(
                        list.find_element(by='css selector', value='.custom-rcaview-action-review').text
                    ),
                    decision = list.find_element(by='css selector', value='.col-md-12 b').text.replace('Action Taken: ', '')
                )
                event_minutes_items.append(model)
            except (selenium.common.exceptions.NoSuchElementException):
                continue
    except (selenium.common.exceptions.NoSuchElementException):
        continue
driver.quit()

C:\Users\MSI_NB\AppData\Local\Temp\ipykernel_27012\3243052260.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


We are currently in the website: https://lims.minneapolismn.gov/File/2022-00585
The togglet context: Collapse Legislative History
Expand Legislative History
Approved
1.
Passage of Ordinance amending Title 14, Chapter 362 of the Minneapolis Code of Ordinances relating to Liquor and Beer: Liquor Licenses, amending the provision to align off-sale malt liquor packaging requirements for brewers with newly-enacted state statute.
decision: Approved
name: This link open a new window
2022-027
title: 1.
Passage of Ordinance amending Title 14, Chapter 362 of the Minneapolis Code of Ordinances relating to Liquor and Beer: Liquor Licenses, amending the provision to align off-sale malt liquor packaging requirements for brewers with newly-enacted state statute.
matter type: Effective
date parsed!
date parsed!
date parsed!
date parsed!
We got the matched vote date: 2022-06-30 00:00:00
clicking into the voting tab
start finding voters!
adding into event_minutes_items!
Go out!
--------------------------

TimeoutException: Message: 


In [125]:
len(event_minutes_items)

87

In [88]:
type(city_council.name)

pandas.core.series.Series

In [ ]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
driver.get('https://lims.minneapolismn.gov/File/2022-00559')

